In [2]:
# Instalación de las bilbiotecas y librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import os

Cargamos los datos MNIST

In [3]:
# Descarga de datos y normalización
(X_train, _), (_, _) = mnist.load_data()
X_train = X_train / 255.0 * 2 - 1  # Normalizar a [-1, 1]
X_train = np.expand_dims(X_train, axis=3)

11490434/11490434 [==============================] - 1s 0us/step


Creamos el generador de imagenes

In [4]:
def build_generator(z_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=z_dim))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(28 * 28 * 1, activation='tanh'))
    model.add(Reshape((28, 28, 1)))
    return model

Creamos el discriminador

In [5]:
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1, activation='sigmoid'))
    return model

Creamos la GAN

In [6]:
def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [7]:
# Construir y compilar el discriminador
discriminator = build_discriminator(X_train.shape[1:])
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [8]:
# Construir el generador
generator = build_generator(100)

In [9]:
# Mantener el discriminador no entrenable cuando se entrena la GAN
discriminator.trainable = False

In [10]:
# Construir y compilar la GAN
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

Entranmos la GAN

In [11]:
losses = []
accuracies = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval):
    for iteration in range(iterations):
        # ----------- Entrenar el Discriminador -----------
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(z)

        d_loss_real = discriminator.train_on_batch(imgs, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((batch_size, 1)))
        d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ----------- Entrenar el Generador -----------
        z = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(z, np.ones((batch_size, 1)))

        if (iteration + 1) % sample_interval == 0:
            losses.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration + 1)

            print(f"Iteración {iteration + 1}, [D pérdida: {d_loss}, precisión: {100.0 * accuracy}%], [G pérdida: {g_loss}]")
    discriminator.save('/Users/n.c.rodriguez/nico/curso_AI/IA Generativa/MNIST GAN/discriminator_gan_mnist.keras')
    generator.save('/Users/n.c.rodriguez/nico/curso_AI/IA Generativa/MNIST GAN/generator_gan_mnist.keras')

In [12]:
def generate_images(generator, image_count, z_dim):
    z = np.random.normal(0, 1, (image_count, z_dim))
    gen_imgs = generator.predict(z)
    gen_imgs = 0.5 * gen_imgs + 0.5  # Escalar las imágenes a [0, 1]

    fig, axs = plt.subplots(image_count, 1, figsize=(10, 10))
    cnt = 0
    for i in range(image_count):
        axs[i].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
        axs[i].axis('off')
        cnt += 1
    plt.show()

In [ ]:
if (os.path.exists('/Users/n.c.rodriguez/nico/curso_AI/IA Generativa/MNIST GAN/generator_gan_mnist.keras')):
    # Cargar el Generador
    generator = load_model('/Users/n.c.rodriguez/nico/curso_AI/IA Generativa/MNIST GAN/generator_gan_mnist.keras')
    # Cargar el Discriminador
    # discriminator = load_model('discriminator_gan_mnist.keras')
    # Generar y Mostrar Imágenes
    generate_images(generator, 5, 100)  # Generar 5 imágenes
else:
    # Parámetros
    iterations = 10000
    batch_size = 128
    sample_interval = 1000
    # Parámetros
    # iterations = 20000  # Aumentar las iteraciones
    # batch_size = 64     # Reducir el tamaño del lote
    # sample_interval = 500  # Muestras más frecuentes
    # Entrenamiento
    train(iterations, batch_size, sample_interval)

4/4 [==============================] - 0s 540us/step
Iteración 1000, [D pérdida: 0.1152002140879631, precisión: 97.65625%], [G pérdida: 3.3321335315704346]
4/4 [==============================] - 0s 468us/step
Iteración 2000, [D pérdida: 0.10376135632395744, precisión: 95.3125%], [G pérdida: 5.856000900268555]
4/4 [==============================] - 0s 545us/step
Iteración 3000, [D pérdida: 0.16531272581778467, precisión: 96.09375%], [G pérdida: 9.780624389648438]
4/4 [==============================] - 0s 559us/step
Iteración 4000, [D pérdida: 0.08595694601535797, precisión: 96.484375%], [G pérdida: 5.973400115966797]
4/4 [==============================] - 0s 560us/step
Iteración 5000, [D pérdida: 0.3503686487674713, precisión: 89.0625%], [G pérdida: 4.573849678039551]
4/4 [==============================] - 0s 488us/step
Iteración 6000, [D pérdida: 0.10798220708966255, precisión: 96.875%], [G pérdida: 5.581427574157715]
4/4 [==============================] - 0s 529us/step
Iteración 7000,